<a href="https://colab.research.google.com/github/ZetaRR/NY-apartments-EDA-Regression-project/blob/main/GettingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Source Type:** Custom-made Zillow API

Link for API: https://rapidapi.com/apimaker/api/zillow-com1/



---



In [1]:
from google.colab import drive, files
import json 
import pandas as pd
import requests
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

In [2]:
drive.mount('/drive', force_remount=False)

#keys =
city = 'New York'
state = 'NY'
search_query = city + ', ' + state
print(search_query)

Mounted at /drive
New York, NY


In [3]:
api_key_df = pd.read_csv('/drive/MyDrive/df_to_csv/api_key.csv')
api_key = api_key_df.iloc[0,:].apply(str).values
api_key = ' '.join([str(i) for i in api_key]) 

In [4]:
#FIRST PAGE
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"1"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()
#response

In [5]:
pd.set_option('display.max_columns', 500)
df_first_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_first_page.columns)
print('Nuw of Columns -->', len(df_first_page.columns))
print('Num of rows -->', len(df_first_page) )
df_first_page.head(2)

Columns --> Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'daysOnZillow',
       'price', 'listingDateTime', 'longitude', 'latitude',
       'contingentListingType', 'listingStatus', 'zpid', 'imgSrc',
       'livingArea', 'bathrooms', 'lotAreaUnit', 'country', 'currency',
       'bedrooms', 'hasImage', 'listingSubType.is_FSBA',
       'listingSubType.is_openHouse'],
      dtype='object')
Nuw of Columns --> 22
Num of rows --> 40


,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse
0,None,APARTMENT,NaN,"474 48th Ave APT 29B, Long Island City, NY 11109",-1,710000,None,-73.95634,40.744408,None,FOR_SALE,79739498,https://photos.zillowstatic.com/fp/93f19aea44f...,1046.0,2,None,USA,USD,2,True,True,NaN
1,None,APARTMENT,NaN,"4-74 48th Ave PENTHOUSE 1C, Long Island City, ...",-1,555555,None,-73.95634,40.744408,None,FOR_SALE,219836786,https://photos.zillowstatic.com/fp/df7f9142c3c...,921.0,1,None,USA,USD,1,True,True,NaN


In [ ]:
#EXTENDED SEARCH
zpids = df_first_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
details_cols = [
      'propertyTaxRate',
      'timeOnZillow',
      'isListedByOwner',
      'zipcode',
      'annualHomeownersInsurance',
      'rentZestimate',
      'yearBuilt',
      'pageViewCount',
      'listingProvider.postingGroupName',
      'resoFacts.cityRegion',
      'resoFacts.hasCooling',
      'resoFacts.hasView']

df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_first_page = pd.concat([df_first_page, df_prop_details], axis=1)
df_first_page.head(5)

In [ ]:
#SECOND PAGE
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"2"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()
#response

In [ ]:
df_second_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_second_page.columns)
print('Nuw of Columns -->', len(df_second_page.columns))
print('Num of rows -->', len(df_second_page) )
df_second_page.head(5)

In [ ]:
#EXTENDED SEARCH
zpids = df_second_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_second_page = pd.concat([df_second_page, df_prop_details], axis=1)
df_second_page.head(5)

In [ ]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"3"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
pd.set_option('display.max_columns', 500)
df_third_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_third_page.columns)
print('Nuw of Columns -->', len(df_third_page.columns))
print('Num of rows -->', len(df_third_page) )
df_third_page.head(5)

In [ ]:
zpids = df_third_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_third_page = pd.concat([df_third_page, df_prop_details], axis=1)
df_third_page.head(5)

In [ ]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"4"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
pd.set_option('display.max_columns', 500)
df_fourth_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_fourth_page.columns)
print('Nuw of Columns -->', len(df_fourth_page.columns))
print('Num of rows -->', len(df_fourth_page) )
df_fourth_page.head(5)

In [ ]:
zpids = df_fourth_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_fourth_page = pd.concat([df_fourth_page, df_prop_details], axis=1)
df_third_page.head(5)

In [ ]:
#concating all dataframes
df = pd.concat([df_first_page, df_second_page, df_third_page, df_fourth_page],axis=0)

In [ ]:
df.to_csv('/drive/MyDrive/df_to_csv/zillow_df.csv', encoding='utf-8', index=False) # saving df on my own Google Drive